In [ ]:
!pip install claudette

In [21]:
import os
import httpx
from claudette import *
from google.colab import userdata
import re
import json
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')

In [3]:
url = "https://raw.githubusercontent.com/vishalbakshi/fastbook-benchmark/refs/heads/main/fastbook-benchmark.json"
response = httpx.get(url)
data = response.json()
len(data['questions']), data['questions'][0]

(191,
 {'chapter': 1,
  'question_number': 1,
  'question_text': 'Do you need these for deep learning?\n\n- Lots of math T / F\n   - Lots of data T / F\n   - Lots of expensive computers T / F\n   - A PhD T / F',
  'gold_standard_answer': '"Lots of math - False\nLots of data - False\nLots of expensive computers - False\nA PhD - False"',
  'answer_context': [{'answer_component': '"Lots of math - False\nLots of data - False\nLots of expensive computers - False\nA PhD - False"',
    'scoring_type': 'simple',
    'context': ['```asciidoc\n[[myths]]\n.What you don\'t need to do deep learning\n[options="header"]\n|======\n| Myth (don\'t need) | Truth\n| Lots of math | Just high school math is sufficient\n| Lots of data | We\'ve seen record-breaking results with <50 items of data\n| Lots of expensive computers | You can get what you need for state of the art work for free\n|======\n```'],
    'explicit_context': 'true',
    'extraneous_answer': 'false'}],
  'question_context': []})

In [4]:
prompt = """<instruction>Decompose the original text into separate components without changing any language. Each component should be a unique concept that would require its own unique definition or explanation. Use the response template shown below. Extract text verbatim from the original text.</instruction>

<original-text>
{text}
<original-text>

<additional-guidance>
- When a sentence contains multiple distinct concepts connected by conjunctions or lists, consider whether each part would require different explanations. If so, separate them into separate components.
- Separate structural facts from their descriptions. "What exists" vs "what it contains/does" are often distinct concepts and should be their own component.
- Each component should represent a single concept that would need its own unique definition or context from source material.
</additional-guidance>

<response-template>
<component id="1">text goes here</component>
<component id="2">text goes here</component>
<component id="...">...</component>
</response-template>

<instruction>Decompose the original text into separate components without changing any language. Each component should be a unique concept that would require its own unique definition or explanation. Use the response template shown above. Extract text verbatim from the original text.</instruction>

<additional-guidance>
- When a sentence contains multiple distinct concepts connected by conjunctions or lists, consider whether each part would require different explanations. If so, separate them into separate components.
- Separate structural facts from their descriptions. "What exists" vs "what it contains/does" are often distinct concepts and should be their own component.
- Each component should represent a single concept that would need its own unique definition or context from source material.
</additional-guidance>
"""

print(prompt.format(text=data['questions'][4]['gold_standard_answer']))

<instruction>Decompose the original text into separate components without changing any language. Each component should be a unique concept that would require its own unique definition or explanation. Use the response template shown below. Extract text verbatim from the original text.</instruction>

<original-text>
"In 1969, Marvin Minsky and Seymour Papert demonstrated in their book, “Perceptrons”, that a single layer of artificial neurons cannot learn simple, critical mathematical functions like XOR logic gate. While they subsequently demonstrated in the same book that additional layers can solve this problem, only the first insight was recognized, leading to the start of the first AI winter.

In the 1980’s, models with two layers were being explored. Theoretically, it is possible to approximate any mathematical function using two layers of artificial neurons. However, in practices, these networks were too big and too slow. While it was demonstrated that adding additional layers impro

In [5]:
model = "claude-3-5-haiku-20241022"
model

'claude-3-5-haiku-20241022'

In [11]:
chat = Chat(model, sp="""You are a helpful and concise assistant.""")
chat("I'm Vishal")

Hi Vishal! How can I help you today?

<details>

- id: `msg_01SGki33UFx3zsArEhFXRfgZ`
- content: `[{'citations': None, 'text': 'Hi Vishal! How can I help you today?', 'type': 'text'}]`
- model: `claude-3-5-haiku-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 21, 'output_tokens': 15, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [24]:
ch4 = [item for item in data['questions'] if item['chapter'] == 4]
ch4[0]

{'chapter': 4,
 'question_number': 2,
 'question_text': '"How are the files and folders in the MNIST_SAMPLE dataset structured? Why?"',
 'gold_standard_answer': '"There are two subfolders, train and valid, the former contains the data for model training, the latter contains the data for validating model performance after each training step. Evaluating the model on the validation set serves two purposes: a) to report a human-interpretable metric such as accuracy (in contrast to the often abstract loss functions used for training), b) to facilitate the detection of overfitting by evaluating the model on a dataset it hasn’t been trained on (in short, an overfitting model performs increasingly well on the training set but decreasingly so on the validation set). Of course, every practicioner could generate their own train/validation-split of the data. Public datasets are usually pre-split to simplifiy comparing results between implementations/publications.\n\nEach subfolder has two subsubfo

In [25]:
ch4[5]['gold_standard_answer']

'"Root mean square error (RMSE), also called the L2 norm, and mean absolute difference (MAE), also called the L1 norm, are two commonly used methods of measuring “distance”. Simple differences do not work because some difference are positive and others are negative, canceling each other out. Therefore, a function that focuses on the magnitudes of the differences is needed to properly measure distances. The simplest would be to add the absolute values of the differences, which is what MAE is. RMSE takes the mean of the square (makes everything positive) and then takes the square root (undoes squaring)."'

In [26]:
chat = Chat(model, sp="""You are a helpful and concise assistant.""")
res = chat(prompt.format(text=ch4[5]['gold_standard_answer']))
print(res.content[0].text)

<component id="1">Root mean square error (RMSE), also called the L2 norm</component>
<component id="2">mean absolute difference (MAE), also called the L1 norm</component>
<component id="3">two commonly used methods of measuring "distance"</component>
<component id="4">Simple differences do not work because some difference are positive and others are negative, canceling each other out</component>
<component id="5">a function that focuses on the magnitudes of the differences is needed to properly measure distances</component>
<component id="6">The simplest would be to add the absolute values of the differences, which is what MAE is</component>
<component id="7">RMSE takes the mean of the square (makes everything positive) and then takes the square root (undoes squaring)</component>


In [27]:
for o in re.finditer(r'<component id="\d+">(.*?)</component>', res.content[0].text): print(o)

<re.Match object; span=(0, 84), match='<component id="1">Root mean square error (RMSE), >
<re.Match object; span=(85, 170), match='<component id="2">mean absolute difference (MAE),>
<re.Match object; span=(171, 250), match='<component id="3">two commonly used methods of me>
<re.Match object; span=(251, 398), match='<component id="4">Simple differences do not work >
<re.Match object; span=(399, 529), match='<component id="5">a function that focuses on the >
<re.Match object; span=(530, 649), match='<component id="6">The simplest would be to add th>
<re.Match object; span=(650, 790), match='<component id="7">RMSE takes the mean of the squa>


In [28]:
[match.group(1) for match in re.finditer(r'<component id="\d+">(.*?)</component>', res.content[0].text)]

['Root mean square error (RMSE), also called the L2 norm',
 'mean absolute difference (MAE), also called the L1 norm',
 'two commonly used methods of measuring "distance"',
 'Simple differences do not work because some difference are positive and others are negative, canceling each other out',
 'a function that focuses on the magnitudes of the differences is needed to properly measure distances',
 'The simplest would be to add the absolute values of the differences, which is what MAE is',
 'RMSE takes the mean of the square (makes everything positive) and then takes the square root (undoes squaring)']

In [29]:
all_components = []
for o in ch4[:5]:
    chat = Chat(model, sp="""You are a helpful and concise assistant.""")
    res = chat(prompt.format(text=o['gold_standard_answer']))
    components = [match.group(1) for match in re.finditer(r'<component id="\d+">(.*?)</component>', res.content[0].text)]
    all_components.append(components)

In [30]:
all_components

[['There are two subfolders, train and valid',
  'the former contains the data for model training',
  'the latter contains the data for validating model performance after each training step',
  'Evaluating the model on the validation set serves two purposes: a) to report a human-interpretable metric such as accuracy (in contrast to the often abstract loss functions used for training)',
  "b) to facilitate the detection of overfitting by evaluating the model on a dataset it hasn't been trained on (in short, an overfitting model performs increasingly well on the training set but decreasingly so on the validation set)",
  'every practicioner could generate their own train/validation-split of the data',
  'Public datasets are usually pre-split to simplifiy comparing results between implementations/publications',
  'Each subfolder has two subsubfolders 3 and 7 which contain the .jpg files for the respective class of images',
  'This is a common way of organizing datasets comprised of pictur

In [31]:
evaluations = []
for i,o in enumerate(ch4[:5]):
    item = {
        "chapter": o["chapter"],
        "question_number": o["question_number"],
        "question_text": o["question_text"],
        "gold_standard_answer": o["gold_standard_answer"],
        "ground_truth_components": [x['answer_component'] for x in o['answer_context']],
        "haiku_components": all_components[i]
        }
    evaluations.append(item)

In [32]:
evaluations

[{'chapter': 4,
  'question_number': 2,
  'question_text': '"How are the files and folders in the MNIST_SAMPLE dataset structured? Why?"',
  'gold_standard_answer': '"There are two subfolders, train and valid, the former contains the data for model training, the latter contains the data for validating model performance after each training step. Evaluating the model on the validation set serves two purposes: a) to report a human-interpretable metric such as accuracy (in contrast to the often abstract loss functions used for training), b) to facilitate the detection of overfitting by evaluating the model on a dataset it hasn’t been trained on (in short, an overfitting model performs increasingly well on the training set but decreasingly so on the validation set). Of course, every practicioner could generate their own train/validation-split of the data. Public datasets are usually pre-split to simplifiy comparing results between implementations/publications.\n\nEach subfolder has two subs

In [33]:
with open('evals.json', 'w') as f: json.dump(evaluations, f, indent=2)

In [34]:
all_components = []
for o in ch4:
    chat = Chat(model, sp="""You are a helpful and concise assistant.""")
    res = chat(prompt.format(text=o['gold_standard_answer']))
    components = [match.group(1) for match in re.finditer(r'<component id="\d+">(.*?)</component>', res.content[0].text)]
    all_components.append(components)

In [35]:
evaluations = []
for i,o in enumerate(ch4):
    item = {
        "chapter": o["chapter"],
        "question_number": o["question_number"],
        "question_text": o["question_text"],
        "gold_standard_answer": o["gold_standard_answer"],
        "ground_truth_components": [x['answer_component'] for x in o['answer_context']],
        "haiku_components": all_components[i]
        }
    evaluations.append(item)

In [36]:
with open('evals.json', 'w') as f: json.dump(evaluations, f, indent=2)